 # Part 1:  Video Upload and Frame Extraction

1. Install the Required Libraries

In [1]:
import cv2
import os

2. Directory for video uplaod and frame extraction

In [2]:

# Function to create directory for storing frames
def create_frame_dir(video_file):
    frame_dir = os.path.splitext(video_file)[0] + "_frames"
    if not os.path.exists(frame_dir):
        os.makedirs(frame_dir)
    return frame_dir

# Function to extract and save frames from the video
def extract_frames(video_file):
    # Open the video file
    cap = cv2.VideoCapture(video_file)
    
    # Create directory to store the frames
    frame_dir = create_frame_dir(video_file)
    
    frame_count = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break  # If no frame is captured, exit
        
        # Save frame as image
        frame_path = os.path.join(frame_dir, f"frame_{frame_count}.jpg")
        cv2.imwrite(frame_path, frame)
        
        frame_count += 1
        print(f"Extracted frame {frame_count}")
    
    cap.release()
    print(f"Frame extraction completed. Frames are saved in {frame_dir}")

# Call the function with your video file path
video_file = "your_video.mp4"  # Replace with your video file
extract_frames(video_file)


Frame extraction completed. Frames are saved in your_video_frames


# Part 2: Player Detection Using YOLOv5

Step 1: Install YOLOv5

In [5]:
import torch
from pathlib import Path

ModuleNotFoundError: No module named 'torch'

Step 2: Use YOLOv5 to detect players in frames.

In [8]:

# Load YOLOv5 pre-trained model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

# Detect players in the extracted frames
def detect_players_in_frames(frame_dir):
    frame_paths = list(Path(frame_dir).glob('*.jpg'))
    
    for frame_path in frame_paths:
        # Perform detection
        results = model(frame_path)
        
        # Display results (bounding boxes around detected players)
        results.show()  # This will show the image with detection

        # If you want to save the results, uncomment the following line
        # results.save()  # This will save images with bounding boxes to the current directory
        
        print(f"Detected players in {frame_path}")

# Call the function with the frame directory
frame_dir = "your_video_frames"  # Replace with your frame directory
detect_players_in_frames(frame_dir)


Using cache found in C:\Users\dhira/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2024-10-5 Python-3.12.3 torch-2.4.1+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


# Part 3: Tracking the Player Across Frames

Step 1: Install DeepSORT

In [9]:
import numpy as np
import sklearn

print("NumPy version:", np.__version__)
print("scikit-learn version:", sklearn.__version__)



NumPy version: 1.26.0
scikit-learn version: 1.5.2


In [13]:
import sys

In [17]:
from deep_sort_realtime.deepsort_tracker import DeepSort


Step 2: Player tracking with DeepSORT

In [18]:

# Initialize the DeepSORT object
deepsort = DeepSort('ckpt.t7')  # You need the pre-trained checkpoint model

# Function to track players across frames
def track_players_in_video(video_file, frame_dir):
    cap = cv2.VideoCapture(video_file)
    frame_count = 0
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break  # Exit if no frame is captured

        # Perform detection (similar to part 2)
        results = model(frame)
        bboxes = results.xyxy[0][:, :4].cpu().numpy()  # Bounding boxes
        confs = results.xyxy[0][:, 4].cpu().numpy()    # Confidence scores

        # Perform tracking
        outputs = deepsort.update(bboxes, confs, frame)

        # Draw the bounding boxes and track IDs on the frame
        for output in outputs:
            x1, y1, x2, y2, track_id = output[:5]
            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (255, 0, 0), 2)
            cv2.putText(frame, f"ID: {track_id}", (int(x1), int(y1)-10), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (255, 0, 0), 2)

        cv2.imshow("Tracking", frame)
        frame_count += 1
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# Call the tracking function
track_players_in_video(video_file, frame_dir)


# Part 4: Pose Estimation Using OpenPose

Step 1: Install OpenPose

In [2]:
import cv2
import mediapipe as mp

Step 2: Implement Pose Estimation on Extracted Frames

In [4]:

# Initialize MediaPipe Pose module
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5)

# Function to perform pose estimation on a single frame
def pose_estimation_on_frame(frame):
    # Convert frame to RGB as MediaPipe works with RGB images
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Perform pose estimation
    results = pose.process(rgb_frame)

    # Draw keypoints on the frame
    annotated_frame = frame.copy()
    if results.pose_landmarks:
        mp.solutions.drawing_utils.draw_landmarks(annotated_frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
    
    return annotated_frame, results.pose_landmarks

# Process all frames for pose estimation
def process_pose_estimation_on_video(video_file):
    cap = cv2.VideoCapture(video_file)

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        # Perform pose estimation
        annotated_frame, landmarks = pose_estimation_on_frame(frame)

        # Display the result
        cv2.imshow("Pose Estimation", annotated_frame)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()

# Call the pose estimation function with your video
video_file = 'path_to_your_video.mp4'  # Specify the path to your video
process_pose_estimation_on_video(video_file)


# Part 5: Action Recognition

Step 1: Define Actions and Recognition Logic

In [5]:
# Part 5: Action Recognition

def recognize_action(landmarks):
    if landmarks is not None:
        # Example: Get key points
        left_shoulder = landmarks.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER]
        left_elbow = landmarks.landmark[mp_pose.PoseLandmark.LEFT_ELBOW]
        left_wrist = landmarks.landmark[mp_pose.PoseLandmark.LEFT_WRIST]

        # Define simple thresholds for action recognition
        shoulder_angle = calculate_angle(
            (left_shoulder.x, left_shoulder.y),
            (left_elbow.x, left_elbow.y),
            (left_wrist.x, left_wrist.y)
        )

        if shoulder_angle < 45:  # This is just an example threshold
            return "Action: Shooting"
        elif shoulder_angle > 135:
            return "Action: Running"
    
    return "Action: Unknown"

# Modify the video processing function to include action recognition
def process_pose_estimation_on_video_with_action_recognition(video_file):
    cap = cv2.VideoCapture(video_file)

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        # Perform pose estimation
        annotated_frame, landmarks = pose_estimation_on_frame(frame)

        # Recognize action
        action = recognize_action(landmarks)
        print(action)

        # Display the result
        cv2.imshow("Pose Estimation", annotated_frame)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()

# Call the action recognition function with your video
process_pose_estimation_on_video_with_action_recognition(video_file)


# Part 6: Generating Feedback for Players

Step 1: Generate Feedback Based on Recognized Actions

In [6]:
# Part 6: Generating Feedback for Players

def generate_feedback(action):
    feedback = ""
    
    if action == "Action: Shooting":
        feedback = "Feedback: Focus on improving your shooting technique. Aim for accuracy."
    elif action == "Action: Running":
        feedback = "Feedback: Maintain a steady pace and good form while running."
    else:
        feedback = "Feedback: Keep practicing your actions for better performance."
    
    return feedback

# Modify the video processing function to include feedback generation
def process_pose_estimation_on_video_with_feedback(video_file):
    cap = cv2.VideoCapture(video_file)

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        # Perform pose estimation
        annotated_frame, landmarks = pose_estimation_on_frame(frame)

        # Recognize action
        action = recognize_action(landmarks)
        print(action)

        # Generate feedback
        feedback = generate_feedback(action)
        print(feedback)

        # Display the result
        cv2.imshow("Pose Estimation", annotated_frame)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()

# Call the feedback generation function with your video
process_pose_estimation_on_video_with_feedback(video_file)


# Now Simple Implementation of getting video and anlysis them

1. Testing and Validation

In [8]:
def analyze_video(video_path):
    # Placeholder for video analysis logic
    # Replace this with actual video analysis code
    # For now, let's just return a dummy feedback
    return f"Analysis completed for {video_path}"

def test_videos(video_list):
    results = []
    for video_path in video_list:
        feedback = analyze_video(video_path)  # Call the video analysis function
        results.append((video_path, feedback))
    return results

# Sample usage
video_list = ['video1.mp4', 'video2.mp4', 'video3.mp4']
results = test_videos(video_list)
for video, feedback in results:
    print(f"Feedback for {video}: {feedback}")



Feedback for video1.mp4: Analysis completed for video1.mp4
Feedback for video2.mp4: Analysis completed for video2.mp4
Feedback for video3.mp4: Analysis completed for video3.mp4


In [9]:
action_threshold = 0.5  # Initial threshold

def set_action_threshold(new_threshold):
    global action_threshold
    action_threshold = new_threshold

# Example of updating the threshold
set_action_threshold(0.6)


2. Data Logging

In [10]:
import csv

def log_feedback(video_path, actions, feedback):
    with open('feedback_log.csv', mode='a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow([video_path, actions, feedback])

# Sample logging
log_feedback('video1.mp4', 'Running, Passing', 'Improve passing accuracy')


In [11]:
import sqlite3

def initialize_db():
    conn = sqlite3.connect('player_analysis.db')
    cursor = conn.cursor()
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS performance (
            id INTEGER PRIMARY KEY,
            video_path TEXT,
            actions TEXT,
            feedback TEXT
        )
    ''')
    conn.commit()
    conn.close()

def insert_performance(video_path, actions, feedback):
    conn = sqlite3.connect('player_analysis.db')
    cursor = conn.cursor()
    cursor.execute('''
        INSERT INTO performance (video_path, actions, feedback) 
        VALUES (?, ?, ?)
    ''', (video_path, actions, feedback))
    conn.commit()
    conn.close()

# Initialize the database
initialize_db()

# Insert sample performance
insert_performance('video1.mp4', 'Running, Passing', 'Improve passing accuracy')


3. User Interface

In [12]:
import tkinter as tk
from tkinter import filedialog, messagebox

def upload_video():
    video_path = filedialog.askopenfilename()
    if video_path:
        feedback = analyze_video(video_path)  # Replace with your video analysis function
        messagebox.showinfo("Feedback", feedback)

root = tk.Tk()
root.title("Player Performance Analysis")

upload_button = tk.Button(root, text="Upload Video", command=upload_video)
upload_button.pack(pady=20)

root.mainloop()


In [14]:
# from flask import Flask, request, render_template

# app = Flask(__name__)

# @app.route('/')
# def home():
#     return render_template('index.html')

# @app.route('/upload', methods=['POST'])
# def upload():
#     if 'video' not in request.files:
#         return "No file uploaded", 400
#     video = request.files['video']
#     video_path = f'uploads/{video.filename}'
#     video.save(video_path)
#     feedback = analyze_video(video_path)  # Replace with your analysis function
#     return feedback

# if __name__ == '__main__':
#     app.run(debug=True)


4. Model Improvements

In [16]:
import tensorflow as tf
from tensorflow.keras import layers, models

def create_model(input_shape, num_classes):
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Sample usage
input_shape = (64, 64, 3)  # Example input shape
num_classes = 10  # Set the number of classes (e.g., for 10 classes)
model = create_model(input_shape, num_classes)

# Print model summary to verify the architecture
model.summary()



Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_1 (Conv2D)               │ (None, 62, 62, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 31, 31, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 30752)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │     1,968,192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 10)             │           650 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,969,738 (7.51 MB)

 Trainable params: 1,969,738 (7.51 MB)

 Non-trainable params: 0 (0.00 B)